In [5]:
from src import db
import yfinance as yf

with db.SessionContext() as session:
    
    for meta in db.models.Meta.query(session=session).all():
        
        source = db.models.Source.query(meta_id = meta.id).one_or_none()
        
        if source.source == "NOTSET": continue
        
        if source.source == "yahoo":
            print(meta.ticker)
            ticker = yf.Ticker(meta.ticker)
            hist = ticker.history(period="max", auto_adjust=False)
            hist.columns = hist.columns.str.replace(" ", "_")
            hist.columns = hist.columns.str.lower()
            hist['open'] = hist['open'].round(2)
            hist['high'] = hist['high'].round(2)
            hist['low'] = hist['low'].round(2)
            hist['close'] = hist['close'].round(2)
            hist['tot_return'] = hist['close'].add(hist['dividends']).pct_change().fillna(0).round(5)
            print(hist[hist.dividends != 0])
            break

AAXJ
             open   high    low  close  adj_close   volume  dividends  \
Date                                                                    
2009-06-23  42.04  42.42  41.46  42.09  32.771549   212000      0.195   
2009-12-22  54.35  54.42  54.00  54.18  42.581676   129500      0.507   
2010-06-23  55.94  55.94  54.30  54.64  43.223476  1714300      0.355   
2010-12-21  62.70  62.92  62.54  62.79  50.158077   186800      0.608   
2011-06-22  60.15  60.20  59.73  59.73  48.237659   319600      0.660   
2011-12-20  49.07  49.79  49.03  49.69  40.455883   310800      0.391   
2012-06-21  52.28  52.28  50.87  50.93  41.864986  1458400      0.508   
2012-12-18  59.27  59.58  58.96  59.53  49.380867   679800      0.541   
2013-06-27  54.05  54.59  54.05  54.45  45.601883   381400      0.516   
2013-12-18  59.11  60.19  58.47  59.83  50.554852   526800      0.525   
2013-12-27  59.59  59.66  59.45  59.45  50.255096   641900      0.025   
2014-06-25  61.98  62.20  61.90  62.11  52.928

In [11]:
import inspect
import pandas as pd
from src import db
from src.config import DATABASE_EXC

print(DATABASE_EXC)
# db.client.conn.create_all()

with db.SessionContext() as session:
    for name, obj in inspect.getmembers(db.models):
        
        if inspect.isclass(obj):
            print(obj)
            tablename = obj.__tablename__
            try:
                records = pd.read_excel(DATABASE_EXC, sheet_name=tablename)
            except:
                continue
            obj.update(records=records, session=session)
            print(f"update {name} complete")

        
        

c:\Users\hanti\Codes\RobertLab\src/database/database.xlsx
<class 'src.database.models.meta.Equity'>
<class 'src.database.models.meta.Index'>
<class 'src.database.models.meta.Meta'>
update Meta complete
<class 'src.database.models.meta.Source'>
update Source complete
<class 'src.database.models.meta.Strategy'>
<class 'src.database.models.meta.Universe'>


In [1]:

from src.core import metrics

import yfinance as yf

prices = yf.download("SPY, AGG")["Adj Close"]

start = metrics.to_starts(prices)
end = metrics.to_ends(prices)
end - start

[*********************100%***********************]  2 of 2 completed


AGG    7119 days
SPY   11014 days
dtype: timedelta64[ns]

In [ ]:
import glob
from datetime import date, timedelta
import numpy as np
import pandas as pd

path = "price/*.xls"
files = glob.glob(pathname=path)

yesterday = date.today() - timedelta(days=1)

for file in files:
    data = pd.read_excel(file)[["종목", "현재가"]]
    data.columns = ["ticker", "value"]
    if pd.api.types.is_object_dtype(data["value"].dtype):
        data["value"] = data["value"].str.replace(",", "").astype(float)
    data["date"] = yesterday
    data["field"] = "CLOSE"
    print(data)


In [ ]:
from src.core import strategy
from src.core import metrics
import yfinance as yf

prices = yf.download("XLU, XLK, XLB, XLP, XLY, XLI, XLV, XLF")["Adj Close"]

strategy = strategy.Momentum(prices).simulate(
    start="2015-1-1", min_volatility=0.12
)

strategy.value.plot()
print(strategy.analytics())
